In [30]:
import numpy as np
import scipy.linalg
import tabulate

In [31]:


def _check_square(A):
    assert len(A.shape) == 2, "only 2d matrices are supported"
    assert A.shape[0] == A.shape[1], "only nxn matrices are supported"


def lu_doolittle(A):
    _check_square(A)

    n = A.shape[0]

    U = np.zeros((n, n), dtype=np.double)
    L = np.eye(n, dtype=np.double)

    with np.errstate(divide="ignore", invalid="ignore"):
        for k in range(n):
            U[k, k:] = A[k, k:] - L[k, :k] @ U[:k, k:]
            L[(k+1):, k] = (A[(k+1):, k] - L[(k+1):, :] @ U[:, k]) / U[k, k]

        L[~np.isfinite(L)] = 0

    return L, U



In [32]:


def noisy_matrix(n):
    np.random.seed(59005)

    matrix = -np.random.choice(5, size=(n, n)).astype(np.double)
    for i in range(n):
        matrix[i, i] = -(np.sum(matrix[i]) - matrix[i, i]) + 10 ** -n

    return matrix


def hilbert_generator(n): return np.fromfunction(lambda i, j: 1 / (i + j + 1),
                                                 (n, n), dtype=np.float)  # since we are starting from i=0 and j=0



In [33]:


def test_lu(A):
    L, U = lu_doolittle(A)
    A_1 = L @ U
    error_1 = np.linalg.norm(A - A_1)

    P, L, U = LU_FUNC(A)
    error_2 = np.linalg.norm(A - P @ L @ U)

    return error_1, error_2


LU_FUNC = scipy.linalg.lu


LU_TESTS = [
    ("LU Decomposition", test_lu, (
        (np.array([[1, 0, 3], [0, 3, 1], [0, 0, 6]]),),
        (np.array([[1, 4, 5], [6, 8, 22], [32, 5, 5]]),),
        (noisy_matrix(5),),
        (noisy_matrix(7),),
        (noisy_matrix(9),),
        (noisy_matrix(11),),
        (noisy_matrix(13),),
        (hilbert_generator(5),),
        (hilbert_generator(7),),
        (hilbert_generator(9),),
        (hilbert_generator(11),),
        (hilbert_generator(13),)
    ))
]


for name, func, args in LU_TESTS:
    results = []
    print(f"TEST {name}")
    for A in args:
        results.append(func(*A))
    print(tabulate.tabulate(results, headers=["error_my", "error_lib"]))
    print("="*80)
    print()



TEST LU Decomposition
   error_my    error_lib
-----------  -----------
0            0
0            4.44089e-16
1.22933e-15  1.35064e-15
2.16022e-15  2.38931e-15
4.04169e-15  3.42961e-15
6.98658e-15  7.66177e-15
5.2079e-15   5.81255e-15
4.16334e-17  2.77556e-17
4.80741e-17  5.19259e-17
7.07631e-17  6.245e-17
8.35553e-17  7.75792e-17
8.63885e-17  8.35553e-17



In [34]:


def forward_substitution(L, b):
    _check_square(L)
    assert L.shape[0] == b.shape[0]

    n = L.shape[0]
    y = np.zeros_like(b, dtype=np.double)

    y[0] = b[0] / L[0, 0]

    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]

    return y


def back_substitution(U, y):
    _check_square(U)
    assert U.shape[0] == y.shape[0]

    n = U.shape[0]
    x = np.zeros_like(y, dtype=np.double)

    x[-1] = y[-1] / U[-1, -1]

    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i:], x[i:])) / U[i, i]

    return x


def lu_solve(A, b):
    assert A.shape[0] == b.shape[0]

    L, U = lu_doolittle(A)

    y = forward_substitution(L, b)

    return back_substitution(U, y)



In [35]:


def test_solve(A, b):
    x = lu_solve(A, b)
    error_my = np.linalg.norm(A @ x - b)
    error_lib = np.linalg.norm(A @ scipy.linalg.solve(A, b) - b)

    return error_my, error_lib


SOLVE_TESTS = [
    ("LU Solve", test_solve, (
        (
            (np.array([[1, 4, 5], [6, 8, 22], [32, 5, 5]]),
             np.array([1, 2, 3.]))
        ),
        (
            (noisy_matrix(5), np.dot(noisy_matrix(5), np.arange(1, 6)))
        ),
        (
            (noisy_matrix(7), np.dot(noisy_matrix(7), np.arange(1, 8)))
        ),
        (
            (noisy_matrix(9), np.dot(noisy_matrix(9), np.arange(1, 10)))
        ),
        (
            (noisy_matrix(11), np.dot(noisy_matrix(11), np.arange(1, 12)))
        ),
        (
            (noisy_matrix(13), np.dot(noisy_matrix(13), np.arange(1, 14)))
        ),
        (
            (hilbert_generator(5), np.dot(hilbert_generator(5), np.arange(1, 6)))
        ),
        (
            (hilbert_generator(7), np.dot(hilbert_generator(7), np.arange(1, 8)))
        ),
        (
            (hilbert_generator(9), np.dot(hilbert_generator(9), np.arange(1, 10)))
        ),
        (
            (hilbert_generator(11), np.dot(hilbert_generator(11), np.arange(1, 12)))
        ),
        (
            (hilbert_generator(13), np.dot(hilbert_generator(13), np.arange(1, 14)))
        )
    ))
]


for name, func, args in SOLVE_TESTS:
    results = []
    print(f"TEST {name}")
    for A in args:
        results.append(func(*A))
    print(tabulate.tabulate(results, headers=["error_my", "error_lib"]))
    print("="*80)
    print()



TEST LU Solve
   error_my    error_lib
-----------  -----------
0            0
7.32411e-15  8.56528e-15
2.16103e-14  1.42109e-14
4.43734e-14  2.30926e-14
6.47335e-14  7.14087e-14
9.9476e-14   1.01734e-13
6.28037e-16  0
4.44089e-16  4.44089e-16
1.53837e-15  1.25607e-15
2.80867e-15  2.03507e-15
7.5886e-15   3.76822e-15



In [36]:


def lu_inverse(A):
    _check_square(A)

    n = A.shape[0]

    b = np.eye(n, dtype=np.double)
    A_inv = np.zeros_like(A, dtype=np.double)

    L, U = lu_doolittle(A)

    for i in range(n):
        y = forward_substitution(L, b[:, i])
        A_inv[:, i] = back_substitution(U, y)

    return A_inv



In [37]:


def test_inv(A):
    inv = lu_inverse(A)
    # High mistake on small numbers
    # inv_lib = scipy.linalg.inv(A)
    inv_lib = np.linalg.inv(A)

    I = np.eye(A.shape[0])
    error_my = np.linalg.norm(A @ inv - I)
    error_lib = np.linalg.norm(A @ inv_lib - I)
    
    return error_my, error_lib

INV_TESTS = [
    ("LU Inverse", test_inv, (
        (np.array([[1, 0, 3], [0, 3, 1], [0, 0, 6]]),),
        (np.array([[1, 4, 5], [6, 8, 22], [32, 5, 5]]),),
        (noisy_matrix(5),),
        (noisy_matrix(7),),
        (noisy_matrix(9),),
        (noisy_matrix(11),),
        (noisy_matrix(13),),
        (hilbert_generator(5),),
        (hilbert_generator(7),),
        (hilbert_generator(9),),
        (hilbert_generator(11),),
        (hilbert_generator(13),)
    ))
]


for name, func, args in INV_TESTS:
    results = []
    print(f"TEST {name}")
    for A in args:
        results.append(func(*A))
    print(tabulate.tabulate(results, headers=["error_my", "error_lib"]))
    print("="*80)
    print()



TEST LU Inverse
   error_my    error_lib
-----------  -----------
2.77556e-17  2.77556e-17
2.32253e-15  3.14631e-16
9.44627e-11  9.00566e-11
1.26059e-08  1.91538e-08
2.00362e-06  2.39538e-06
0.000338649  0.000275188
0.0321888    0.0340475
8.25478e-12  1.05584e-11
3.74552e-09  8.23867e-09
6.36571e-06  4.92949e-06
0.00554747   0.00580983
7.09535      4.03951

